In [390]:
import pandas as pd
import numpy as np
from bertopic import BERTopic #expects list of strings
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
import os

In [108]:
with open("../../data/climate_classified/climate_classified_posts_2.json") as f:
    df = pd.read_json(f)
df.head()
df_og = df


In [ ]:
#generate data for testing
data = df[df["label"] == "yes"]
data = data[data["score"]>= 0.99]
data_text = data["text"]


In [4]:
sub_dfs = []
for i in np.arange(10000,110000,10000):
    sub_dfs.append(df_og[i-10000:i])
len(sub_dfs)

10

In [ ]:
filtered_sub_dfs = []
for i in np.arange(5000,30000,5000):
    filtered_sub_dfs.append(filtered_df[i-5000:i])
len(filtered_sub_dfs)

5

In [65]:
# List of posts (strings)
climate_posts = filtered_sub_dfs[0]["text"]
# climate_posts = climate_posts[:3200]

In [ ]:
# Load your Bluesky posts (climate-labeled ones)
# List of 120,000 posts (strings)
# Use a lightweight embedding model (can try 'all-MiniLM-L6-v2' first)
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

umap_model = UMAP(n_neighbors=8, n_components=7, min_dist=0.02, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=8, prediction_data=True)

# Create BERTopic instance
topic_model = BERTopic(
    embedding_model=embedding_model, 
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    verbose=True)


In [381]:

# Fit the model
topics, probs = topic_model.fit_transform(filtered_df_text)

# Visualize topics
# topic_model.visualize_topics()

2025-04-22 04:29:38,941 - BERTopic - Embedding - Transforming documents to embeddings.


Batches: 100%|██████████| 26/26 [00:28<00:00,  1.08s/it]
2025-04-22 04:30:07,125 - BERTopic - Embedding - Completed ✓
2025-04-22 04:30:07,127 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-22 04:30:10,692 - BERTopic - Dimensionality - Completed ✓
2025-04-22 04:30:10,695 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-22 04:30:10,802 - BERTopic - Cluster - Completed ✓
2025-04-22 04:30:10,833 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-22 04:30:10,973 - BERTopic - Representation - Completed ✓


In [382]:
topic_model.get_topic_info().head(100)

,Topic,Count,Name,Representation,Representative_Docs
0,0,811,0_the_to_and_of,"[the, to, and, of, in, is, it, that, for, are]",[update I forgot to post the other day I ended...
1,1,10,1_gorgeous_artsky_comdp0989433404_captivatesnyt,"[gorgeous, artsky, comdp0989433404, captivates...",[TINY-MIGHTY-TURTLE! Gorgeous watercolors Kirk...


In [389]:
# Create a DataFrame to explore the topics easily
df = pd.DataFrame({"text": filtered_df_text, "topic": topics})

# Filter to see texts assigned to topic 3
topic_3_texts = df[df["topic"] == 1]

# Show the first few
pd.set_option('display.max_colwidth', None)
print(topic_3_texts["text"].head(10))
pd.reset_option('display.max_colwidth')

3859     TINY-MIGHTY-TURTLE! Gorgeous watercolors Kirkus SweetHuffP CaptivatesNYT OnomatopoeiaPW Courageous SFGate TranquilALA SerenadesNatGeo amazon.comdp0989433404 kidlit writerslift books art nature ocean kids WritingCommunity BookSky ArtSky illustration climate turtles
9071     TINY-MIGHTY-TURTLE! Gorgeous watercolors Kirkus SweetHuffP CaptivatesNYT OnomatopoeiaPW Courageous SFGate TranquilALA SerenadesNatGeo amazon.comdp0989433404 kidlit writerslift books art nature ocean kids WritingCommunity BookSky ArtSky illustration climate turtles
12834    TINY-MIGHTY-TURTLE! Gorgeous watercolors Kirkus SweetHuffP CaptivatesNYT OnomatopoeiaPW Courageous SFGate TranquilALA SerenadesNatGeo amazon.comdp0989433404 kidlit writerslift books art nature ocean kids WritingCommunity BookSky ArtSky illustration climate turtles
16633    TINY-MIGHTY-TURTLE! Gorgeous watercolors Kirkus SweetHuffP CaptivatesNYT OnomatopoeiaPW Courageous SFGate TranquilALA SerenadesNatGeo amazon.comdp0989433404 kidlit write

In [ ]:
#can reduce the toppic size down to nr_topics
topic_model.reduce_topics(filtered_df_text, nr_topics=6)
topic_model.get_topic_info().head(100)

ValueError: All arrays must be of the same length

In [391]:
os.getcwd()

'c:\\Users\\abels\\ITU\\BachProj\\DS_BachelorProject_PH\\notebooks\\Classification'

In [402]:
data_path = "../../data/climate_classified"

def prepare_data(df):
    data = df[df["label"] == "yes"]
    data = data[data["score"]>= 0.99]
    data_text = data["text"]
    return data, data_text

#generates BERT instance
def generate_model(neighbors, components, distance, cluster):
    # Use a lightweight embedding model (can try 'all-MiniLM-L6-v2' first)
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

    umap_model = UMAP(n_neighbors=neighbors, n_components=components, min_dist=distance, metric='cosine', random_state=42)
    hdbscan_model = HDBSCAN(min_cluster_size=cluster, prediction_data=True)

    # Create BERTopic instance
    topic_model = BERTopic(
        embedding_model=embedding_model, 
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        verbose=True)
    return topic_model

def main():
    for filename in os.listdir(data_path):
        print(f"\nworking on: {filename}\n")
        df = pd.read_json(f"{data_path}/{filename}")
        filtered_df, filtered_text = prepare_data(df)
        topic_model = generate_model(neighbors=8,components=5,distance=0.01,cluster=8)
        topics, probs = topic_model.fit_transform(filtered_text)
        print(topic_model.get_topic_info())
    
if __name__ == "__main__":
    main()


working on: climate_classified_posts_113.json



2025-04-22 17:27:02,559 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 32/32 [00:29<00:00,  1.10it/s]
2025-04-22 17:27:31,654 - BERTopic - Embedding - Completed ✓
2025-04-22 17:27:31,655 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-22 17:27:35,993 - BERTopic - Dimensionality - Completed ✓
2025-04-22 17:27:35,994 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-22 17:27:36,072 - BERTopic - Cluster - Completed ✓
2025-04-22 17:27:36,076 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-22 17:27:36,211 - BERTopic - Representation - Completed ✓


    Topic  Count                                    Name  \
0      -1    380                        -1_the_and_to_in   
1       0     76                        0_of_to_the_they   
2       1     68                 1_climate_change_the_is   
3       2     42                2_climate_the_and_social   
4       3     35       3_weather_temperature_pressure_at   
5       4     31              4_credits_energy_tax_clean   
6       5     29                     5_coal_to_wind_cost   
7       6     25     6_shipping_carbon_international_the   
8       7     23                    7_apr_snow_precip_14   
9       8     22          8_fossil_gas_eusign2025_wemove   
10      9     21               9_could_vets_veteran_ohio   
11     10     20       10_schools_cars_transport_greener   
12     11     20         11_vote_canada_climate_election   
13     12     19                       12_ai_and_need_it   
14     13     18                   13_zero_net_bbc_steel   
15     14     15               14_china_

2025-04-22 17:27:39,794 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 32/32 [00:25<00:00,  1.26it/s]
2025-04-22 17:28:05,322 - BERTopic - Embedding - Completed ✓
2025-04-22 17:28:05,323 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-22 17:28:08,555 - BERTopic - Dimensionality - Completed ✓
2025-04-22 17:28:08,556 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-22 17:28:08,609 - BERTopic - Cluster - Completed ✓
2025-04-22 17:28:08,612 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-22 17:28:08,780 - BERTopic - Representation - Completed ✓


    Topic  Count                                           Name  \
0      -1    339                               -1_the_to_of_and   
1       0     56                      0_biodiversity_the_and_to   
2       1     53                        1_bsky_social_and_april   
3       2     44                       2_water_plastic_for_more   
4       3     42                      3_europe_climate_2024_the   
5       4     41                         4_climate_and_the_that   
6       5     30                                5_ai_it_and_use   
7       6     30                              6_co2_04_14_space   
8       7     27       7_trump_environmental_administration_epa   
9       8     25                          8_oil_prices_will_gas   
10      9     25                       9_weather_rain_0mph_wind   
11     10     25                          10_apr_snow_15_precip   
12     11     24                        11_electric_ev_evs_cars   
13     12     24        12_crisis_climate_bsky_apannierrunache

...

   Topic  Count                             Name  \
0      0    519                  0_the_to_of_and   
1      1    128         1_apr_missing_precip_low   
2      2    107        2_airport_apr_snow_precip   
3      3     60          3_snow_depth_apr_precip   
4      4      9  4_issues_excessive_043z_outlook   

                                      Representation  \
0      [the, to, of, and, is, in, it, that, for, on]   
1  [apr, missing, precip, low, high, snow, climat...   
2  [airport, apr, snow, precip, low, high, missin...   
3  [snow, depth, apr, precip, low, high, climate,...   
4  [issues, excessive, 043z, outlook, wpc, rainfa...   

                                 Representative_Docs  
0  [This is the future Pierre Poilievre, you're t...  
1  [BURNS OR Apr 6 Climate Report High 69 Low 26 ...  
2  [STOCKTON AIRPORT CA Apr 6 Climate Report High...  
3  [LAREDO Apr 6 Climate Report High 69 Low 50 Pr...  
4  [WPC issues Day 1 Moderate Risk Excessive Rain...  

working on: climate

2025-04-22 17:33:00,157 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 28/28 [00:16<00:00,  1.68it/s]
2025-04-22 17:33:16,866 - BERTopic - Embedding - Completed ✓
2025-04-22 17:33:16,867 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-22 17:33:20,185 - BERTopic - Dimensionality - Completed ✓
2025-04-22 17:33:20,186 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-22 17:33:20,236 - BERTopic - Cluster - Completed ✓
2025-04-22 17:33:20,238 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-22 17:33:20,338 - BERTopic - Representation - Completed ✓


    Topic  Count                              Name  \
0      -1     19                 -1_link_pm_apr_12   
1       0    529                   0_the_to_and_of   
2       1    108             1_airport_apr_12_snow   
3       2     64               2_snow_depth_apr_12   
4       3     60        3_link_missing_precip_high   
5       4     41                 4_apr_sun_13_0000   
6       5     24                 5_sat_12_apr_0000   
7       6     14        6_wind_is_pressure_weather   
8       7     11       7_trace_link_missing_precip   
9       8     11             8_1473_sol_mean_local   
10      9     10  9_avoidance_hazard_camera_hazcam   

                                       Representation  \
0   [link, pm, apr, 12, issues, az, grassland, mes...   
1      [the, to, and, of, it, in, is, for, that, you]   
2   [airport, apr, 12, snow, precip, high, low, mi...   
3   [snow, depth, apr, 12, precip, high, low, clim...   
4   [link, missing, precip, high, low, 12, snow, a...   
5   [apr,

2025-04-22 17:33:22,337 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 3/3 [00:02<00:00,  1.34it/s]
2025-04-22 17:33:24,581 - BERTopic - Embedding - Completed ✓
2025-04-22 17:33:24,582 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-22 17:33:24,831 - BERTopic - Dimensionality - Completed ✓
2025-04-22 17:33:24,832 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-22 17:33:24,836 - BERTopic - Cluster - Completed ✓
2025-04-22 17:33:24,839 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-22 17:33:24,869 - BERTopic - Representation - Completed ✓


   Topic  Count                    Name  \
0     -1     27   -1_the_to_climate_and   
1      0     47         0_the_to_and_in   
2      1     10  1_you_to_huge_negative   

                                      Representation  \
0  [the, to, climate, and, in, about, with, that,...   
1      [the, to, and, in, of, is, it, that, for, we]   
2  [you, to, huge, negative, carbon, my, no, for,...   

                                 Representative_Docs  
0  [Climate Scale will attend the WindEurope Annu...  
1  [This is a good point by sabrinafernandes.bsky...  
2  [I make no apologies for my huge carbon footpr...  


In [ ]:
original_df topics


array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.     